In [1]:
# from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1664277218219_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load data from s3 buckets

In [2]:
input_data = "s3a://udacity-dend/"
output_data = "s3://my-sparkify123/sparkify/"
log_data = input_data +"log_data/*/*/*.json"
song_data = input_data+"song_data/*/*/*/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dfSong = spark.read.json(song_data)
dfSong.printSchema()
dfSong.createOrReplaceTempView("stagingsongs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [4]:
dfSong.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist_id='AR4T2IF1187B9ADBB7', artist_latitude=63.96027, artist_location='<a href="http://billyidol.net" onmousedown=\'UntrustedLink.bootstrap($(this), "fc44f8f60d13ab68c56b3c6709c6d670", event)\' target="_blank" rel="nofollow">http://billyidol.net</a>', artist_longitude=10.22442, artist_name='Billy Idol', duration=233.22077, num_songs=1, song_id='SOVIYJY12AF72A4B00', title='The Dead Next Door (Digitally Remastered 99)', year=1983)]

In [5]:
dfLog = spark.read.json(log_data)
dfLog.printSchema()
dfLog.createOrReplaceTempView("stagingevents")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [6]:
dfLog.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26')]

# Create Songs Table

In [4]:
songs_table = spark.sql(""" SELECT  DISTINCT song_id,
                                             title,
                                             artist_id,
                                             year,
                                             duration
                            FROM stagingsongs
                            WHERE song_id IS NOT NULL
                        """)  
songs_table.printSchema()
songs_table.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

[Row(song_id='SOLLALT12A8C1399F3', title='Piano Concerto No.2 In F Major Op.102 (1957) : III Allegro', artist_id='ARWMEJW11F4C83C123', year=0, duration=319.37261)]

In [5]:
songs_table.write.partitionBy("year","artist_id").parquet(output_data + "songs.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Artists Table

In [6]:
artists_table = spark.sql("""
                                SELECT  DISTINCT artist_id AS artist_id,
                                        artist_name As name,
                                        artist_location AS location,
                                        artist_latitude AS latitude,
                                        artist_longitude AS longitude
                                FROM stagingsongs
                                WHERE artist_id IS NOT NULL   
                          """)
artists_table.printSchema()
artists_table.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

[Row(artist_id='AR7DQY51187B9B2D88', name='Chingo Bling w/ Baby Bash', location='Texas', latitude=31.1689, longitude=-100.07715), Row(artist_id='AROOKC21187B9AE85C', name='Missy Higgins', location='Melbourne, Australia', latitude=-37.81753, longitude=144.96715), Row(artist_id='AR4YSKY1187B9A2B88', name='Kind of Like Spitting', location='Portland, OR', latitude=45.51179, longitude=-122.67563), Row(artist_id='ARB57BN1187B9B5EAF', name='36 Crazyfists', location='Anchorage, AK', latitude=61.21756, longitude=-149.85776), Row(artist_id='AR9DO2O1187FB53637', name='ColdWorld', location='Germany (Erfurt, Thuringia)', latitude=50.97768, longitude=11.02307)]

In [7]:
artists_table.write.parquet(output_data + "artists.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Users Table

In [9]:
users_table = spark.sql("""
                            SELECT  DISTINCT userId AS user_id,
                                    firstName AS first_name,
                                    lastName AS last_name,
                                    gender AS gender,
                                    level AS level
                            FROM stagingevents
                            WHERE userId IS NOT NULL
                            AND page = 'NextSong'
                            """)
users_table.printSchema()
users_table.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

[Row(user_id='98', first_name='Jordyn', last_name='Powell', gender='F', level='free'), Row(user_id='34', first_name='Evelin', last_name='Ayala', gender='F', level='free'), Row(user_id='85', first_name='Kinsley', last_name='Young', gender='F', level='paid'), Row(user_id='38', first_name='Gianna', last_name='Jones', gender='F', level='free'), Row(user_id='85', first_name='Kinsley', last_name='Young', gender='F', level='free')]

In [11]:
users_table.write.parquet( output_data + "users.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Time Table

In [11]:
from pyspark.sql.types import DateType,TimestampType

@udf(TimestampType())
def get_timestamp(ts):
    return datetime.fromtimestamp(ts/1000.0)

@udf()
def get_datetime(ts):
    return datetime.fromtimestamp(ts/1000.0).strftime('%Y-%m-%d %H:%M:%S')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
dfLog = spark.sql(""" SELECT * From stagingevents WHERE ts IS NOT NULL """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# create timestamp column from original timestamp column
# get_timestamp = udf()
dfLog = dfLog.withColumn("timestamp",get_timestamp("ts"))
dfLog.head(5)                   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp=datetime.datetime(2018, 11, 15, 0, 30, 26, 796000), datetime='2018-11-15 00:30:26'), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.

In [18]:
dfLog.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- datetime: string (nullable = true)

In [19]:
# get_datetime = udf()

dfLog = dfLog.withColumn("datetime",get_datetime("ts"))
dfLog.head(5)                   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp=datetime.datetime(2018, 11, 15, 0, 30, 26, 796000), datetime='2018-11-15 00:30:26'), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.

In [20]:
dfLog.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- datetime: string (nullable = true)

In [21]:
dfLog.createOrReplaceTempView("stagingevents")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
time_table = spark.sql("""SELECT  DISTINCT datetime as start_time,
                                    hour ( timestamp) AS hour,
                                    day ( timestamp) AS day,
                                    weekofyear ( timestamp) AS week,
                                    month ( timestamp) AS month,
                                    year ( timestamp) AS year,
                                    weekday ( timestamp) AS weekday
                            FROM stagingevents
                            WHERE page = 'NextSong'
                        """)
time_table.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(start_time='2018-11-15 07:28:47', hour=7, day=15, week=46, month=11, year=2018, weekday=3), Row(start_time='2018-11-15 16:49:50', hour=16, day=15, week=46, month=11, year=2018, weekday=3), Row(start_time='2018-11-15 17:46:01', hour=17, day=15, week=46, month=11, year=2018, weekday=3), Row(start_time='2018-11-21 07:53:00', hour=7, day=21, week=47, month=11, year=2018, weekday=2), Row(start_time='2018-11-14 18:35:36', hour=18, day=14, week=46, month=11, year=2018, weekday=2)]

In [29]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [30]:
time_table.write.partitionBy("year","month").parquet(output_data + "time.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Songplays Table

In [39]:
from pyspark.sql.functions import monotonically_increasing_id
songplays_table = spark.sql("""
                            SELECT  monotonically_increasing_id() AS songplay_id,
                                    ste.timestamp AS start_time,
                                    month ( timestamp) AS month,
                                    year ( timestamp) AS year,
                                    ste.userId AS user_id,
                                    ste.level AS level,
                                    sts.song_id AS song_id, 
                                    sts.artist_id AS artist_id,
                                    ste.sessionId AS session_id,
                                    ste.location AS location,
                                    ste.userAgent AS user_agent
                            FROM stagingevents ste
                            JOIN stagingsongs sts
                            ON sts.title = ste.song 
                            AND sts.artist_name = ste.artist  
                            AND sts.duration = ste.length
                            WHERE ste.page = 'NextSong'  
""")

songplays_table.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(songplay_id=8589934592, start_time=datetime.datetime(2018, 11, 21, 8, 25, 43, 796000), month=11, year=2018, user_id='88', level='paid', song_id='SOCHPTV12A6BD53113', artist_id='ARN8NCB1187FB49652', session_id=744, location='Sacramento--Roseville--Arden-Arcade, CA', user_agent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'), Row(songplay_id=25769803776, start_time=datetime.datetime(2018, 11, 29, 16, 58, 1, 796000), month=11, year=2018, user_id='49', level='paid', song_id='SOGXSWA12A6D4FBC99', artist_id='ARPFHN61187FB575F6', session_id=1041, location='San Francisco-Oakland-Hayward, CA', user_agent='Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0'), Row(songplay_id=34359738368, start_time=datetime.datetime(2018, 11, 28, 8, 18, 57, 796000), month=11, year=2018, user_id='58', level='paid', song_id='SOJWCWM12A8C13B664', artist_id='ARM6T8I1187FB36CC8', session_id=887, location='Augusta-Richmon

In [40]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)

In [41]:
songplays_table.write.partitionBy("year","month").parquet(output_data + "songplays.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…